In [1]:
using LowLevelFEM

In [2]:
gmsh.initialize()
gmsh.open("body.geo")

Info    : Reading 'body.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.007109s, CPU 0.002094s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Don

In [3]:
mat = material("body")
prob = Problem([mat])

Problem("body", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 561, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
load1 = load("load1", fy=-1, fz=2)
supp = displacementConstraint("supp", ux=0, uy=0, uz=0)

("supp", 0, 0, 0)

In [5]:
u = solveDisplacement(prob, [load1], [supp]);

In [6]:
F = solveStress(u, DoFResults=true);

In [7]:
K1 = stiffnessMatrix(prob);
f1 = loadVector(prob, [load1]);

In [8]:
f_all = K1 * u;

In [9]:
f_T = f_all - f1;

In [10]:
showDoFResults(u, name="displacement")
showDoFResults(f1, name="load")
showDoFResults(f_all, name="total force")
showDoFResults(f_T, name="reaction force")

4

In [11]:
resultant(f1, "body")

3-element Vector{Float64}:
    0.0
 -100.00000000000024
  200.00000000000048

In [12]:
resultant(f_T, "body")

3-element Vector{Float64}:
    1.6919798895287386e-12
  100.00000000000725
 -199.9999999999979

In [13]:
n1 = normalVector(prob, "load1");

In [14]:
F1 = nodesToElements(F, onPhysicalGroup="load1")
p1 = F1 * n1;

In [15]:
ex = VectorField(prob, [field("load1", fx=1, fy=0, fz=0)]);
ey = VectorField(prob, [field("load1", fx=0, fy=1, fz=0)]);
ez = VectorField(prob, [field("load1", fx=0, fy=0, fz=1)]);

In [16]:
f20 = load("load1", fx=p1 * ex, fy=p1 * ey, fz=p1 * ez);
f2 = loadVector(prob, [f20]);

In [17]:
showDoFResults(f2, name="load from BC")

5

In [18]:
resultant(f2, "load1")

3-element Vector{Float64}:
   0.30571481433162123
 -98.07564037691125
 200.08453072707204

In [19]:
integrate(prob, "load1", p1 * ey)

-98.075640376911

In [20]:
resultant(f1 - f2, "load1")

3-element Vector{Float64}:
 -0.30571481433162123
 -1.9243596230889821
 -0.08453072707163134

In [21]:
showDoFResults(f1 - f2, name="load difference")

6

In [22]:
openPostProcessor()

XOpenIM() failed
Fontconfig warning: using without calling FcInit()


-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [23]:
gmsh.finalize()